In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Conv1D, MaxPooling1D, Add
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import pandas as pd
import glob
import librosa

# Функция загрузки полного набора MNIST
def load_full_mnist():
    """
    Загрузка полной выборки MNIST (784 входных признака + класс метки [0..9]).
    Нормировка входных пикселей к диапазону [0, 1].
    """
    df = pd.read_csv('mnist_784.csv')
    X = df.iloc[:, 0:784].values / 255.0
    y = df.iloc[:, 784].values  
    return X, y

# Функция для удаления тишины с начала и конца аудиосигнала
def extract_useful_signal(signal, sr, top_db=20):
    trimmed_signal, idx = librosa.effects.trim(signal, top_db=top_db)
    return trimmed_signal

# Функция загрузки и предварительной обработки аудио
def load_preprocess_audio(num_samples, target_length=5000):
    audio_files = glob.glob('free-spoken-digit-dataset/recordings/*.wav')
    np.random.shuffle(audio_files)
    selected_files = audio_files[:num_samples]
    
    X_audio_list = []
    digits_audio = []

    for file in selected_files:
        base_name = file.split('/')[-1]
        digit_str = base_name.split('_')[0]
        audio_digit = int(digit_str)
        digits_audio.append(audio_digit)
         
        signal, sr = librosa.load(file, sr=None)
        extract = extract_useful_signal(signal, sr, top_db=20)            

        if len(extract) > target_length:
            signal = extract[:target_length]
        else:
            signal = np.pad(extract, (0, target_length - len(extract)), 'constant')
        
        # Масштабирование сигнала к диапазону [0, 1]
        min_val, max_val = np.min(signal), np.max(signal)
        if max_val - min_val < 1e-12:
            signal = np.zeros_like(signal)
        else:
            signal = (signal - min_val) / (max_val - min_val)
        
        X_audio_list.append(signal)
    
    X_audio = np.array(X_audio_list)
    return X_audio, sr, digits_audio

# Функция сопоставления изображений MNIST с аудиоданными на основе метки цифры
def match_images_to_audio_digits(X, y, digits_audio, num_samples):
    matched_images = []
    for d in digits_audio:
        indices = np.where(y == d)[0]
        if len(indices) == 0:
            raise ValueError(f"Не найдено изображений в MNIST с цифрой {d}")
        idx = np.random.choice(indices)
        matched_images.append(X[idx])
    matched_images = np.array(matched_images)
    return matched_images

# ------------------------------
# Подготовка данных для CNN
# ------------------------------
num_samples = 1000

# Загрузка и обработка MNIST
X_full, y_full = load_full_mnist()
# Получение изображений, соответствующих аудиозаписям
X_image = match_images_to_audio_digits(X_full, y_full, 
                                       digits_audio=load_preprocess_audio(num_samples)[2], 
                                       num_samples=num_samples)
# Приведение изображений к форме (28, 28, 1)
X_image = X_image.reshape(-1, 28, 28, 1)

# Загрузка и обработка аудио
X_audio, sr, digits_audio = load_preprocess_audio(num_samples=num_samples, target_length=5000)
# Приведение аудио к форме (5000, 1)
X_audio = X_audio.reshape(-1, 5000, 1)

# Используем метки из аудиоданных (цифры)
y_labels = np.array(digits_audio)

# Разбиение на обучающую и тестовую выборки
X_image_train, X_image_test, X_audio_train, X_audio_test, y_train, y_test = train_test_split(
    X_image, X_audio, y_labels, test_size=0.2, random_state=42, stratify=y_labels)

# ------------------------------
# Определение архитектуры CNN
# ------------------------------
# Ветвь для обработки изображений (MNIST)
image_input = Input(shape=(28, 28, 1), name='image_input')
x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(image_input)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
# Скрытое представление размерности 300
image_hidden = Dense(300, activation='relu', name='image_hidden')(x)

# Ветвь для обработки аудио
audio_input = Input(shape=(5000, 1), name='audio_input')
y = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(audio_input)
y = MaxPooling1D(pool_size=2)(y)
y = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(y)
y = MaxPooling1D(pool_size=2)(y)
y = Flatten()(y)
# Скрытое представление размерности 300
audio_hidden = Dense(300, activation='relu', name='audio_hidden')(y)

# Объединение представлений (операция суммирования)
combined_hidden = Add(name='combined_hidden')([image_hidden, audio_hidden])

# Финальный классификатор
output = Dense(10, activation='softmax', name='output')(combined_hidden)

model_cnn = Model(inputs=[image_input, audio_input], outputs=output)
model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_cnn.summary()

# ------------------------------
# Обучение модели
# ------------------------------
history = model_cnn.fit([X_image_train, X_audio_train], y_train, 
                         epochs=10, batch_size=32, validation_split=0.2)

# ------------------------------
# Оценка классификации
# ------------------------------
test_loss, test_accuracy = model_cnn.evaluate([X_image_test, X_audio_test], y_test)
print("Точность на тестовой выборке: {:.2f}%".format(test_accuracy * 100))

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 audio_input (InputLayer)       [(None, 5000, 1)]    0           []                               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 28, 28, 32)   320         ['image_input[0][0]']            
                                                                                                  
 conv1d_2 (Conv1D)              (None, 5000, 32)     128         ['audio_input[0][0]']            
                                                                                            